# Future formats: NGFF/Zarr

The OME team is an effort to design a new cloud-friendly “Next Generation” file format (NGFF), combining the meta-data prowess of OME-TIFF with effecient n-Dimensional file stores.
The current implementation uses Zarr to represent the pixel data as small n-Dimensional chunks.
Zarr is intended to be very similar to the HDF5 format, but with several concessions to make it friendly for cloud-based storage.


Links:
* [zarr-python](https://zarr.readthedocs.io/en/stable/)
* [NGFF specification](https://ngff.openmicroscopy.org)

In [ ]:
# One time only -- install the libraries via pip
!pip install ome-zarr==0.0.17 omero-cli-zarr==0.0.9 zarr==2.6.1

In [ ]:
# General setup for this notebook
import zarr
import matplotlib.pyplot as plt

## Access images from the IDR
The [IDR](https://idr.openmicroscopy.org) is a public repository of images powered my OMERO.

In addition to an OMERO interface, the IDR has all of its images stored in the NGFF format in cloud storage.

We will play around with a zebra fish image, which can be [viewed in a browser](https://hms-dbmi.github.io/vizarr/?source=https://s3.embassy.ebi.ac.uk/idr/zarr/v0.1/4495402.zarr) using the web-base viewer vizarr.

As a programmer, one of the major benefits of Zarr is the array-like interface to the image data.
The `ome_zarr` library provides a useful util for getting information on an image.

In [ ]:
# The ome-zarr package installes the ome_zarr command-line utility for working with ome-zarr (NGFF)
!ome_zarr info https://s3.embassy.ebi.ac.uk/idr/zarr/v0.1/4495402.zarr

Well, let's try to open the Zarr in python. Note, above, that this image is being stored at multiple resolutions!
We'll open the smallest one for now...


In [ ]:
root = zarr.open("https://s3.embassy.ebi.ac.uk/idr/zarr/v0.1/4495402.zarr/")
s7 = root['7']
print(s7.shape)
plt.imshow(s7[0,0,0,:,:])

Ok, that's nice, but what if I am really intrested in a small cutout at high resolution??

In [ ]:
s0 = root['0']
crop = s0[0,0,0,200000:201000,800000:801000]
plt.imshow(crop)

## Extra: Converting OMERO images to zarr

The [omero-cli-zarr](https://github.com/ome/omero-cli-zarr/) can be used to convert an image from OMERO into a local (to you) Zarr file for easy access.
If available, this tool will use the [bioformats2raw](https://github.com/glencoesoftware/bioformats2raw) tool, but will fall back to use the slower pixels service if necessary.

_Optional:_ `conda install -c ome bioformats2raw` is the easiest way to install `bioformats2raw`, but you'll end up installing an entire copy of java as well!

In [ ]:
# Log into the OMERO server using the CLI
# This caches a credential for ~10 minutes for command line access
!omero login --server ctomerodev.jax.org  --user USERNAME --password PASSWORD

In [ ]:
# Download one of the images & check the contents
!omero zarr export Image:6100
!ome_zarr info 6100.zarr

Now, let's open this image and plot all of the Z layers.

In [ ]:
root = zarr.open("6100.zarr")
s7 = root['0']
print(s7.shape)

cols = 4
axes = []
fig = plt.figure(figsize=(16, 10))
for z in range(s7.shape[0]):
    axes.append( fig.add_subplot((s7.shape[0] // cols)+1, cols, z+1))
    subplot_title=("z=%d" % z)
    axes[-1].set_title(subplot_title) 
    plt.imshow(s7[z,0,0,:,:])
plt.show()